In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../')

from typing import List,Dict
from elmo_on_md.data_loaders.sentiment_loader import SentimentLoader
from elmo_on_md.evaluation.sentiment_analysis import SentimentAnalysis
from elmo_on_md.evaluation.model_loader import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
loader = SentimentLoader()
sentiment_data = loader.load_data()

sentences = sentiment_data['train']['sentences']
labels = sentiment_data['train']['labels']

tokens_train,tokens_test, labels_train,labels_test = train_test_split(sentences, labels, test_size=0.2, random_state=1)
train_set = {'sentences':tokens_train,'labels':labels_train}
validate_set = {'sentences':tokens_test,'labels':labels_test}

In [3]:
elmo = load_model('original', batch_size = 32)
sentiment = SentimentAnalysis(elmo,lr=1e-4)

sentiment.train(train_set,validate_set,n_epochs=10, batch_size = 64)

2019-08-20 21:20:11,132 INFO: char embedding size: 2289
2019-08-20 21:20:11,800 INFO: word embedding size: 189561
2019-08-20 21:20:16,503 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(189561, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2289, 50, padding_idx=2286)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

Epoch: 0	 Train Loss: 1.136554479598999	 Validation Loss: 1.1011409759521484


C:\Users\Brus\Anaconda3\envs\elmo\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 1	 Train Loss: 1.0595306158065796	 Validation Loss: 1.111438274383545
Epoch: 2	 Train Loss: 0.26941439509391785	 Validation Loss: 1.3273097276687622
Epoch: 3	 Train Loss: 0.10214462876319885	 Validation Loss: 2.058777332305908
Epoch: 4	 Train Loss: 0.08270316570997238	 Validation Loss: 2.651628017425537
Epoch: 5	 Train Loss: 0.08819936215877533	 Validation Loss: 3.092129707336426
Epoch: 6	 Train Loss: 0.11393529176712036	 Validation Loss: 3.095299482345581
Epoch: 7	 Train Loss: 0.11191695183515549	 Validation Loss: 3.01635479927063
Epoch: 8	 Train Loss: 0.11466176062822342	 Validation Loss: 3.7429888248443604
Epoch: 9	 Train Loss: 0.11769984662532806	 Validation Loss: 3.951070785522461


#### results

In [5]:
# train_preds = sentiment.predict(sentiment_data['train'])
# print(confusion_matrix(sentiment_data['train']['labels'],train_preds))
y_pred = sentiment.predict(sentiment_data['test'])
y_true = sentiment_data['test']['labels']

print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true, y_pred, target_names=['positive','negative','neutral']))

2019-08-20 21:23:33,343 INFO: 80 batches, avg len: 22.8


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 8.00 GiB total capacity; 3.80 GiB already allocated; 1.97 MiB free; 2.18 GiB cached)

# Our Model

In [5]:
elmo = load_model('BiLSTM_pos_weight_8', batch_size = 32)
sentiment = SentimentAnalysis(elmo,lr=1e-4)

sentiment.train(train_set,validate_set,n_epochs=10, batch_size = 64)

2019-08-18 20:49:40,338 INFO: 129 batches, avg len: 16.9
2019-08-18 20:49:42,451 INFO: Finished 1000 sentences.
2019-08-18 20:49:44,790 INFO: Finished 2000 sentences.
2019-08-18 20:49:46,905 INFO: Finished 3000 sentences.
2019-08-18 20:49:49,709 INFO: Finished 4000 sentences.
2019-08-18 20:49:52,556 INFO: Finished 5000 sentences.
2019-08-18 20:49:55,223 INFO: Finished 6000 sentences.
2019-08-18 20:49:57,137 INFO: Finished 7000 sentences.
2019-08-18 20:49:58,926 INFO: Finished 8000 sentences.
2019-08-18 20:50:03,671 INFO: 33 batches, avg len: 17.2
2019-08-18 20:50:06,312 INFO: Finished 1000 sentences.
2019-08-18 20:50:08,802 INFO: Finished 2000 sentences.


Epoch: 0	 Train Loss: 1.1369311809539795	 Validation Loss: 1.0659725666046143
Epoch: 1	 Train Loss: 1.2584692239761353	 Validation Loss: 1.0501151084899902
Epoch: 2	 Train Loss: 1.3318028450012207	 Validation Loss: 1.0098891258239746
Epoch: 3	 Train Loss: 1.3556928634643555	 Validation Loss: 1.0070713758468628
Epoch: 4	 Train Loss: 1.3890658617019653	 Validation Loss: 1.0128661394119263
Epoch: 5	 Train Loss: 1.383247971534729	 Validation Loss: 0.9947163462638855
Epoch: 6	 Train Loss: 0.9775118231773376	 Validation Loss: 1.0206685066223145
Epoch: 7	 Train Loss: 0.8406320214271545	 Validation Loss: 1.0809253454208374
Epoch: 8	 Train Loss: 1.1051946878433228	 Validation Loss: 1.0401324033737183
Epoch: 9	 Train Loss: 1.1153709888458252	 Validation Loss: 1.0313242673873901
Epoch: 10	 Train Loss: 1.0209996700286865	 Validation Loss: 1.0206633806228638
Epoch: 11	 Train Loss: 1.0014313459396362	 Validation Loss: 1.0267301797866821
Epoch: 12	 Train Loss: 0.9209927320480347	 Validation Loss: 1.0

In [7]:
y_pred = sentiment.predict(sentiment_data['test'])
y_true = sentiment_data['test']['labels']

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=['bad','neutral','good']))

2019-08-18 20:51:48,338 INFO: 40 batches, avg len: 17.1
2019-08-18 20:51:51,267 INFO: Finished 1000 sentences.
2019-08-18 20:51:53,262 INFO: Finished 2000 sentences.


[[1408  278   12]
 [ 424  353   13]
 [  43   18   11]]
              precision    recall  f1-score   support

         bad       0.75      0.83      0.79      1698
     neutral       0.54      0.45      0.49       790
        good       0.31      0.15      0.20        72

    accuracy                           0.69      2560
   macro avg       0.53      0.48      0.49      2560
weighted avg       0.67      0.69      0.68      2560

